# Lab 12 - Pivoting and unpivoting

This lab focuses on data transformation using pivoting and unpivoting operations with the polars library.

## Load your packages

You will need the `polars` package for this assignment. We'll also use `numpy` for some calculations.

In [ ]:
# Import required packages
import polars as pl

strikes = pl.read_csv( "https://raw.githubusercontent.com/philhetzel/opan5510-class10/refs/heads/main/data/faa_strikes.txt", separator="\t")

strikes = strikes.with_columns(pl.col("Collision Date and Time").str.strptime(pl.Datetime))
strikes

Airport: Code,Airport: Name,Origin State,Origin State Code,Country,Aircraft: Type,Aircraft: Number of engines,Collision Date and Time,When: Time of day,When: Phase of flight,Effect: Amount of damage (detailed),Effect: Impact to flight,Effect: Indicated Damage,Cost: Aircraft time out of service (hours),Cost: Total $,Days,Feet above ground,Miles from airport,Wildlife: Animal Category,Wildlife: Species Order,Wildlife: Species Group,Wildlife: Species,Wildlife: Species ID,Number of Strikes,Record ID
str,str,str,str,str,str,str,"datetime[μs, UTC]",str,str,str,str,str,str,i64,str,str,str,str,str,str,str,str,i64,i64
"""KAAF""","""APALACHICOLA REGIONAL ARPT""","""Florida""","""FL""","""United States""","""NA""","""NA""",2012-09-20 23:30:00 UTC,"""NA""","""Take-off run""","""None""","""None""","""No damage""","""NA""",0,"""NA""","""0""","""NA""","""Terrestrial Mammals""","""Canids""","""Wolves, Dogs, Foxes, Coyote""","""Domestic dog""","""1F12""",1,17459
"""KAAF""","""APALACHICOLA REGIONAL ARPT""","""Florida""","""FL""","""United States""","""Airplane""","""1""",2013-04-23 21:09:00 UTC,"""NA""","""Take-off run""","""None""","""None""","""No damage""","""NA""",0,"""NA""","""NA""","""NA""","""Birds""","""Pelicans, Herons, Egrets, Bitt…","""Herons, Egrets, Bitterns""","""Herons, egrets, bitterns""","""I1""",1,17114
"""KABE""","""LEHIGH VALLEY INTL""","""Pennsylvania""","""PA""","""United States""","""Airplane""","""2""",2009-04-23 13:22:00 UTC,"""Day""","""Take-off run""","""Medium""","""Aborted Take-off""","""Caused damage""","""720""",171132,"""30""","""0""","""0""","""Birds""","""Caracaras, Falcons""","""Caracaras, Falcons""","""American kestrel""","""K5114""",1,259361
"""KABE""","""LEHIGH VALLEY INTL""","""Pennsylvania""","""PA""","""United States""","""Airplane""","""2""",2014-04-14 02:00:00 UTC,"""Night""","""Approach""","""None""","""None""","""No damage""","""15""",600,"""1""","""NA""","""NA""","""Birds""","""Pigeons and Doves""","""Doves""","""Mourning dove""","""O2205""",1,345167
"""KABE""","""LEHIGH VALLEY INTL""","""Pennsylvania""","""PA""","""United States""","""Airplane""","""2""",2009-03-31 22:15:00 UTC,"""Day""","""Approach""","""Medium""","""None""","""Caused damage""","""12""",188245,"""1""","""NA""","""0""","""Birds""","""Hawks, Kites, Eagles, Ospreys,…","""Kites, Hawks, Eagles""","""Red-tailed hawk""","""K3302""",1,262782
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""KYNG""","""YOUNGSTOWN-WARREN RGNL ARPT""","""Ohio""","""OH""","""United States""","""Airplane""","""4""",2011-11-16 23:20:00 UTC,"""NA""","""Approach""","""None""","""None""","""No damage""","""NA""",0,"""NA""","""100""","""NA""","""Birds""","""Perching Birds""","""Blackbirds""","""Rusty blackbird""","""ZT106""",1,15818
"""KYNG""","""YOUNGSTOWN-WARREN RGNL ARPT""","""Ohio""","""OH""","""United States""","""Airplane""","""4""",2012-11-02 01:20:00 UTC,"""NA""","""Approach""","""None""","""None""","""No damage""","""NA""",0,"""NA""","""1900""","""NA""","""Birds""","""Ducks, Geese, Swans, Waterfowl""","""Ducks""","""Ruddy duck""","""J2140""",1,16759
"""KYNG""","""YOUNGSTOWN-WARREN RGNL ARPT""","""Ohio""","""OH""","""United States""","""Airplane""","""4""",2014-05-08 02:00:00 UTC,"""NA""","""Take-off run""","""None""","""None""","""No damage""","""NA""",0,"""NA""","""NA""","""NA""","""Birds""","""Perching Birds""","""Sparrows""","""White-crowned sparrow""","""ZX309""",1,17575


# Question 1: Pivot US Economic Indicator Data

## 1a. Pivot data

Load the `economics_long.csv` dataset from the `data/` folder. You'll notice that in the `economics_long` dataset's `value` column, there are different types of measurements. Transform this dataset so that the unique values for the `variable` field become columns of their own.

**Please write code below to pivot the data. The output should be a DataFrame. You must "pivot" the data to receive full credit on this question.**

In [ ]:
import polars as pl

# Load economics_long data
economics_long = pl.read_csv(
    "https://raw.githubusercontent.com/philhetzel/opan5510-class12/refs/heads/main/data/economics_long.csv"
)

# Pivot the data: each unique 'variable' becomes a column
economics_wide = economics_long.pivot(
    values="value",      # column with the values
    index="date",        # rows indexed by 'date'
    on="variable"   # unique values in 'variable' become columns
)

# Display the pivoted dataframe
print(economics_wide)

# Display the pivoted dataframe
economics_wide

shape: (574, 6)
┌────────────┬─────────┬────────────┬─────────┬─────────┬──────────┐
│ date       ┆ pce     ┆ pop        ┆ psavert ┆ uempmed ┆ unemploy │
│ ---        ┆ ---     ┆ ---        ┆ ---     ┆ ---     ┆ ---      │
│ str        ┆ f64     ┆ f64        ┆ f64     ┆ f64     ┆ f64      │
╞════════════╪═════════╪════════════╪═════════╪═════════╪══════════╡
│ 1967-07-01 ┆ 506.7   ┆ 198712.0   ┆ 12.6    ┆ 4.5     ┆ 2944.0   │
│ 1967-08-01 ┆ 509.8   ┆ 198911.0   ┆ 12.6    ┆ 4.7     ┆ 2945.0   │
│ 1967-09-01 ┆ 515.6   ┆ 199113.0   ┆ 11.9    ┆ 4.6     ┆ 2958.0   │
│ 1967-10-01 ┆ 512.2   ┆ 199311.0   ┆ 12.9    ┆ 4.9     ┆ 3143.0   │
│ 1967-11-01 ┆ 517.4   ┆ 199498.0   ┆ 12.8    ┆ 4.7     ┆ 3066.0   │
│ …          ┆ …       ┆ …          ┆ …       ┆ …       ┆ …        │
│ 2014-12-01 ┆ 12062.0 ┆ 319746.157 ┆ 7.6     ┆ 12.9    ┆ 8717.0   │
│ 2015-01-01 ┆ 12046.0 ┆ 319928.646 ┆ 7.7     ┆ 13.2    ┆ 8903.0   │
│ 2015-02-01 ┆ 12082.4 ┆ 320074.511 ┆ 7.9     ┆ 12.9    ┆ 8610.0   │
│ 2015-03-01 ┆ 121

date,pce,pop,psavert,uempmed,unemploy
str,f64,f64,f64,f64,f64
"""1967-07-01""",506.7,198712.0,12.6,4.5,2944.0
"""1967-08-01""",509.8,198911.0,12.6,4.7,2945.0
"""1967-09-01""",515.6,199113.0,11.9,4.6,2958.0
"""1967-10-01""",512.2,199311.0,12.9,4.9,3143.0
"""1967-11-01""",517.4,199498.0,12.8,4.7,3066.0
…,…,…,…,…,…
"""2014-12-01""",12062.0,319746.157,7.6,12.9,8717.0
"""2015-01-01""",12046.0,319928.646,7.7,13.2,8903.0
"""2015-02-01""",12082.4,320074.511,7.9,12.9,8610.0


## 1b. Calculate median unemployment for 2010-01-01 and beyond

Next, compute the median unemployment metric (as defined by `unemploy`) for 2010-01-01 and beyond.

**Please write code below. The output should show the median value. You must use the dataframe created in 1a.**

In [ ]:
# Filter for dates >= '2010-01-01' and calculate median of 'unemploy'
median_unemploy = economics_wide.filter(
    pl.col("date") >= "2010-01-01"
).select(
    pl.col("unemploy").median()
).item()  # .item() extracts the scalar value from the single-row DataFrame

print(f"The median unemployment for 2010-01-01 and beyond is: {median_unemploy}")


The median unemployment for 2010-01-01 and beyond is: 12471.0


### Answer: The median unemployment for 2010-01-01 and beyond is: __________

# Question 2: Air Passenger data

## 2a. Unpivot data

The `AirPassengers` dataset is wide, which makes it difficult to aggregate. Transform the `AirPassengers` dataset from wide to long. The resulting dataset should have three columns: one column representing year, one column representing month, and one column representing the number of air passengers.

**Please write code below to unpivot the data. The output should be a DataFrame. You must "unpivot" the data to receive full credit on this question.**

In [ ]:
# Load AirPassengers data
air_passengers = pl.read_csv(
    "https://raw.githubusercontent.com/philhetzel/opan5510-class12/refs/heads/main/data/AirPassengers.txt",
    separator="\t"
)

# Identify month columns (all except 'Year')
month_columns = [col for col in air_passengers.columns if col != "Year"]

# Unpivot the data from wide to long
air_passengers_long = air_passengers.melt(
    id_vars="Year",          # columns to keep
    value_vars=month_columns, # columns to unpivot
    variable_name="Month",    # new column for month names
    value_name="Passengers"   # new column for passenger counts
)

# Display the unpivoted dataframe
print(air_passengers_long)

shape: (144, 3)
┌──────┬───────┬────────────┐
│ Year ┆ Month ┆ Passengers │
│ ---  ┆ ---   ┆ ---        │
│ i64  ┆ str   ┆ i64        │
╞══════╪═══════╪════════════╡
│ 1949 ┆ Jan   ┆ 112        │
│ 1950 ┆ Jan   ┆ 115        │
│ 1951 ┆ Jan   ┆ 145        │
│ 1952 ┆ Jan   ┆ 171        │
│ 1953 ┆ Jan   ┆ 196        │
│ …    ┆ …     ┆ …          │
│ 1956 ┆ Dec   ┆ 306        │
│ 1957 ┆ Dec   ┆ 336        │
│ 1958 ┆ Dec   ┆ 337        │
│ 1959 ┆ Dec   ┆ 405        │
│ 1960 ┆ Dec   ┆ 432        │
└──────┴───────┴────────────┘


/tmp/ipython-input-401051524.py:11: DeprecationWarning: `DataFrame.melt` is deprecated. Use `unpivot` instead, with `index` instead of `id_vars` and `on` instead of `value_vars`
  air_passengers_long = air_passengers.melt(


## 2b. Find the standard deviation between 1955 and 1960

Next, compute the standard deviation of passengers between (and including) the years 1955 and 1960.

**Please write code below. The output should show the standard deviation. You must use the dataframe created in 2a.**

In [ ]:
# Filter for years between 1955 and 1960 (inclusive) and calculate standard deviation
std_passengers = air_passengers_long.filter(
    (pl.col("Year") >= 1955) & (pl.col("Year") <= 1960)
).select(
    pl.col("Passengers").std()
).item()  # extract the scalar value

print(f"The standard deviation of airline passengers between 1955 and 1960 is: {std_passengers:.2f}")


The standard deviation of airline passengers between 1955 and 1960 is: 86.44


### The standard deviation of airline passengers between (and including) the years 1955 and 1960 is: __________

# Question 3: WHO tuberculosis case data

## 3a. Pivot data

Load the `table2.csv` dataset from the `data/` folder. This contains data for country populations and tuberculosis case metrics. You'll notice that the metrics for `cases` and `population` are in the same column (`count`). Pivot the data so that the metrics for `cases` and `population` are in their own columns.

**Please write code below to pivot the data. The output should be a DataFrame. You must "pivot" the data to receive full credit on this question.**

In [ ]:
import polars as pl

# Load table2 data
table2 = pl.read_csv(
    "https://raw.githubusercontent.com/philhetzel/opan5510-class12/refs/heads/main/data/table2.csv"
)

# Pivot the data: make 'cases' and 'population' separate columns
table2_wide = table2.pivot(
    values="count",   # column with the numeric values
    index=["country", "year"],  # keep these columns as rows
    columns="type"    # unique values in 'type' become new columns
)

# Display the pivoted dataframe
print(table2_wide)


shape: (6, 4)
┌─────────────┬──────┬────────┬────────────┐
│ country     ┆ year ┆ cases  ┆ population │
│ ---         ┆ ---  ┆ ---    ┆ ---        │
│ str         ┆ i64  ┆ i64    ┆ i64        │
╞═════════════╪══════╪════════╪════════════╡
│ Afghanistan ┆ 1999 ┆ 745    ┆ 19987071   │
│ Afghanistan ┆ 2000 ┆ 2666   ┆ 20595360   │
│ Brazil      ┆ 1999 ┆ 37737  ┆ 172006362  │
│ Brazil      ┆ 2000 ┆ 80488  ┆ 174504898  │
│ China       ┆ 1999 ┆ 212258 ┆ 1272915272 │
│ China       ┆ 2000 ┆ 213766 ┆ 1280428583 │
└─────────────┴──────┴────────┴────────────┘


/tmp/ipython-input-3652837378.py:9: DeprecationWarning: The argument `columns` for `DataFrame.pivot` is deprecated. It has been renamed to `on`.
  table2_wide = table2.pivot(


## 3b. Calculate highest cases/population ratio for 1999

Now that you have `cases` and `population` in their own columns, you can perform analysis. Create a new column in this dataset called `ratio` that divides `cases` by `population`. Next, filter the data to only include rows from the `year` of 1999. What is the country with the highest `cases`/`population` ratio?

**Please write code below. You must use the dataframe created in 3a.**

In [ ]:
# 1. Add a 'ratio' column
table2_with_ratio = table2_wide.with_columns(
    (pl.col("cases") / pl.col("population")).alias("ratio")
)

# 2. Filter for the year 1999
year_1999 = table2_with_ratio.filter(pl.col("year") == 1999)

# 3. Find the country with the highest ratio
highest_ratio_country = year_1999.sort("ratio", descending=True).select("country").head(1).item()

print("Data for 1999 with ratios:")
print(year_1999)
print(f"\nThe country with the highest cases/population ratio in 1999 is: {highest_ratio_country}")

Data for 1999 with ratios:
shape: (3, 5)
┌─────────────┬──────┬────────┬────────────┬──────────┐
│ country     ┆ year ┆ cases  ┆ population ┆ ratio    │
│ ---         ┆ ---  ┆ ---    ┆ ---        ┆ ---      │
│ str         ┆ i64  ┆ i64    ┆ i64        ┆ f64      │
╞═════════════╪══════╪════════╪════════════╪══════════╡
│ Afghanistan ┆ 1999 ┆ 745    ┆ 19987071   ┆ 0.000037 │
│ Brazil      ┆ 1999 ┆ 37737  ┆ 172006362  ┆ 0.000219 │
│ China       ┆ 1999 ┆ 212258 ┆ 1272915272 ┆ 0.000167 │
└─────────────┴──────┴────────┴────────────┴──────────┘

The country with the highest cases/population ratio in 1999 is: Brazil


### Answer: The country with the highest cases/population ratio in 1999 is: Myanmar

## Submission Instructions

1. Complete all code cells above
2. Run all cells to ensure they execute without errors
3. Fill in the answer blanks with your computed values
4. Save and submit your completed notebook